<a href="https://colab.research.google.com/github/ML-HW-SYS/a3-anya-23-ct/blob/main/5_conv2d_dw_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Depthwise-seperable 2D Convolution on GPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [12]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

# %mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
# !git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

/content/gdrive/MyDrive/ece5545
/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
M	3-conv1d_fpga.ipynb
M	5_conv2d_dw_gpu.ipynb
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 378 bytes | 6.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-anya-23-ct
   a4a1726..de49b70  main       -> origin/main
Updating a4a1726..de49b70
Fast-forward
 src/ops.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1_conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  5_conv2d_dw_gpu.ipynb	README.md  tests
2_conv1d_gpu.ipynb  4_gemm_gpu.ipynb	 leaderboard_id.txt	src


## 2 Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.6/428.6 MB 3.8 MB/s eta 0:00:00


## 3. Implement `make_dwsp_conv2d_gpu_scheduler` function in `src.ops`

In that function, you are required to implemented 2D convolution and use TVM to optimize it.
Please use zero padding and unit stride.
You can assume kernel size to be an odd number.
The padding will equals to kernel size minus ones.
In this case, the output image will preserve the input image dimension.

The `make_dwsp_conv2d_gpu_scheduler` takes following arguments:
1. Batch size $B$;
2. Input channel size $C$;
3. Input image height $H$;
4. Input image width $W$;
5. Output number of channels $O$;
6. Kernel size $K$

You should return both the TVM scheduler and the TVM opterator for
1. Input tensor $x$ with size (B, C, H, W)
2. Input kernel weight $y$ with size (O, 1, K, K)
3. Output $out$ with size (B, O, H, W)

The scheduler should be able to used to build a function with signature $func(x, y, out)$.
Please see the following cells the usage.

In [ ]:
import tvm
import torch.nn.functional as F
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_dwsp_conv2d_gpu_scheduler

B = 3
C = 4
H = 16
W = 32
K = 7
dtype = 'float32'
a_np = np.random.rand(B, C, H, W).astype(dtype)
w_np = np.random.rand(C, 1, K, K).astype(dtype)

s, inp, ker, out = make_dwsp_conv2d_gpu_scheduler(B, C, H, W, K)
func = tvm.build(s, [inp, ker, out], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((B, C, H, W), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)

print("Output:", b)
print(f"2DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Output: [[[[ 3.3085604  5.4952455  5.7319655 ...  8.067335   6.448019
     4.866012 ]
   [ 4.912408   4.6070194  6.2031045 ...  7.691306   6.6890016
     4.5963297]
   [ 4.958172   6.4360023  7.180133  ... 10.454654   8.899951
     6.2019672]
   ...
   [ 7.5075     7.329874   9.383095  ...  7.867528   6.5494347
     3.9976702]
   [ 5.5932813  6.060118   7.592796  ...  6.6397448  5.644244
     3.233382 ]
   [ 4.358755   5.2905345  5.8813543 ...  5.1563425  4.6399117
     2.8146594]]

  [[ 4.354756   5.401951   6.505598  ...  6.046861   4.039886
     3.0361319]
   [ 5.537532   6.2455845  8.215389  ...  6.83536    5.061315
     3.6397796]
   [ 7.345904   7.9559155  9.64554   ...  8.31967    6.487497
     4.533555 ]
   ...
   [ 4.344213   6.1993246  9.180506  ...  7.1866517  6.3072047
     5.4909673]
   [ 4.1907954  5.566031   6.4878144 ...  6.630461   5.6072884
     4.3224173]
   [ 2.301276   4.364054   5.706177  ...  5.4244947  4.317024
     3.717177 ]]

  [[ 5.4084587  5.368762   5.6852

In [ ]:
print(tvm.lower(s, [inp, ker, out], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((3, 4, 16, 32), "float32"), W: T.Buffer((4, 1, 7, 7), "float32"), out: T.Buffer((3, 4, 16, 32), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        pad_in = T.allocate([10032], "float32", "global")
        pad_in_1 = T.Buffer((10032,), data=pad_in)
        for i0, i1 in T.grid(3, 4):
            blockIdx_z = T.launch_thread("blockIdx.z", 11)
            threadIdx_z = T.launch_thread("threadIdx.z", 2)
            blockIdx_y = T.launch_thread("blockIdx.y", 19)
            threadIdx_y = T.launch_thread("threadIdx.y", 2)
            A_1 = T.Buffer((6144,), data=A.data)
            pad_in_1[i0 * 3344 + i1 * 836 + blockIdx_z * 76 + threadIdx_z * 38 + blockIdx_y * 2 + threadIdx_y] = T.if_then_else(blockIdx_z * 2 + threadIdx_z < 3 or 19 <= blockIdx_z * 2 + threadIdx_z or blockIdx_y * 2 + threadIdx_y

In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_dwsp_2dconv_gpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 1357 items                                                                               

tests/test_dwsp_2dconv_gpu.py .............................................................. [  4%]
............................................................................................ [ 11%]
............................................................................................ [ 18%]
............................................................................................ [ 24%]
............................................................................................ [ 31%]
............................................................................................ [ 38%]
...............

After Optimisation

In [13]:
import tvm
import torch.nn.functional as F
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_dwsp_conv2d_gpu_scheduler

B = 3
C = 4
H = 16
W = 32
K = 7
dtype = 'float32'
a_np = np.random.rand(B, C, H, W).astype(dtype)
w_np = np.random.rand(C, 1, K, K).astype(dtype)

s, inp, ker, out = make_dwsp_conv2d_gpu_scheduler(B, C, H, W, K)
func = tvm.build(s, [inp, ker, out], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((B, C, H, W), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)

print("Output:", b)
print(f"2DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Output: [[[[ 5.1544003  5.8808637  8.224626  ...  6.2984886  5.96336
     4.3654714]
   [ 6.8383126  7.5007644  9.456376  ...  7.6629796  7.161763
     4.9138427]
   [ 8.108542   8.747515  11.457553  ... 10.224314   7.4862905
     6.5296154]
   ...
   [ 6.9277368  8.752855  10.285544  ...  8.655933   7.1696286
     5.183069 ]
   [ 6.086805   7.5314403  8.239422  ...  7.282843   6.1466146
     4.3259053]
   [ 4.6753883  4.852292   6.5419607 ...  4.9141173  4.809478
     3.4151871]]

  [[ 4.268148   5.979646   6.6704874 ...  5.3198843  4.8047056
     4.8257375]
   [ 4.0216293  5.849429   7.2215014 ...  6.7832327  5.9706035
     4.366134 ]
   [ 5.9203835  7.623722   9.781009  ...  8.637326   7.7504587
     6.1455   ]
   ...
   [ 5.3856606  6.642796   6.7991385 ...  9.147945   8.207213
     6.609198 ]
   [ 3.6262195  5.48802    5.6210656 ...  8.538963   6.840046
     5.6348567]
   [ 3.743722   3.6602774  5.03133   ...  5.743079   4.907499
     4.5882735]]

  [[ 5.233818   6.268291   6.6232

In [14]:
print(tvm.lower(s, [inp, ker, out], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((3, 4, 16, 32), "float32"), W: T.Buffer((4, 1, 7, 7), "float32"), out: T.Buffer((3, 4, 16, 32), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        pad_in = T.allocate([10032], "float32", "global")
        pad_in_1 = T.Buffer((10032,), data=pad_in)
        for i0, i1 in T.grid(3, 4):
            blockIdx_z = T.launch_thread("blockIdx.z", 6)
            blockIdx_y = T.launch_thread("blockIdx.y", 10)
            threadIdx_z = T.launch_thread("threadIdx.z", 4)
            threadIdx_y = T.launch_thread("threadIdx.y", 4)
            if T.likely(blockIdx_z * 2 + threadIdx_z // 2 < 11):
                if T.likely(blockIdx_y * 2 + threadIdx_y // 2 < 19):
                    A_1 = T.Buffer((6144,), data=A.data)
                    pad_in_1[i0 * 3344 + i1 * 836 + blockIdx_z * 152 + threadIdx_z * 38 +

In [15]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_dwsp_2dconv_gpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 1357 items                                                                               

tests/test_dwsp_2dconv_gpu.py .............................................................. [  4%]
............................................................................................ [ 11%]
............................................................................................ [ 18%]
............................................................................................ [ 24%]
............................................................................................ [ 31%]
............................................................................................ [ 38%]
...............